# Day 16

https://adventofcode.com/2022/day/16


## Part 1

In [1]:
test_data = """\
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""
test_data

'Valve AA has flow rate=0; tunnels lead to valves DD, II, BB\nValve BB has flow rate=13; tunnels lead to valves CC, AA\nValve CC has flow rate=2; tunnels lead to valves DD, BB\nValve DD has flow rate=20; tunnels lead to valves CC, AA, EE\nValve EE has flow rate=3; tunnels lead to valves FF, DD\nValve FF has flow rate=0; tunnels lead to valves EE, GG\nValve GG has flow rate=0; tunnels lead to valves FF, HH\nValve HH has flow rate=22; tunnel leads to valve GG\nValve II has flow rate=0; tunnels lead to valves AA, JJ\nValve JJ has flow rate=21; tunnel leads to valve II'

In [2]:
#import plotly.graph_objects as go
#import numpy as np

In [16]:
data = test_data
valves = {}
for i, line in enumerate(data.split("\n")):

    p1, p2 = line.split(";")
    valve = p1[6:8]
    flow_rate = int(p1.split("=")[-1])
    targets = p2.split(", ")
    targets[0] = targets[0][-2:]
    print(targets)
    valves[valve] = dict(
        flow_rate=flow_rate, 
        targets=targets, 
        opened=False
    )
    if i == 0:
        curr_valve = valve

sol1 = 0
time_left = 30

done = set()

while time_left > 0:
    print(curr_valve)
    flow_rate = valves[curr_valve]["flow_rate"]
    if flow_rate > 0:
        print(f"opening valve, flow rate {flow_rate}" + \
              f"x time left {time_left} = {flow_rate*time_left}")
        time_left -= 1
        sol1 += time_left * flow_rate        
    curr_valve = valves[curr_valve]["targets"]
    time_left -= 1
    


#assert sol1 == 1651
#print("test passed")

['DD', 'II', 'BB']
['CC', 'AA']
['DD', 'BB']
['CC', 'AA', 'EE']
['FF', 'DD']
['EE', 'GG']
['FF', 'HH']
['GG']
['AA', 'JJ']
['II']
AA
DD
opening valve, flow rate 20x time left 29 = 580
CC
opening valve, flow rate 2x time left 27 = 54
DD
opening valve, flow rate 20x time left 25 = 500
AA
II
AA
BB
opening valve, flow rate 13x time left 20 = 260
CC
opening valve, flow rate 2x time left 18 = 36
BB
opening valve, flow rate 13x time left 16 = 208
AA


IndexError: pop from empty list

In [4]:
with open("day16.txt") as f:
    inp_data = f.read()

sol1 = solution1(inp_data, target)
assert sol1 == 5073496
print("test passed")

Result: 5073496
test passed


## Part 2

In [5]:
def solution2(data: str, x_max: int, y_max: int):
    measurements = []

    for line in data.split("\n"):
        try: 
            sensor, beacon = [parse_xy(s) for s in line.split(":")]
        except Exception as e:
            print(line)
            raise e        
        dist = calc_dist(sensor, beacon)
        measurements.append([sensor, dist])

    searched = set()
    for sensor, dist in measurements:
        x0, y0 = sensor
        radius = dist + 1
        #print("Searching at", (x0, y0), "with radius", radius)
        detected = False
        for dx in range(radius + 1):
            dy = radius - dx
            for sgn_x, sgn_y in [(-1, -1), (1, -1), (-1, 1), (1, 1)]:
                x1 = x0 + sgn_x * dx        
                y1 = y0 + sgn_y * dy            
                if not (0 <= x1 <= x_max and 0 <= y1 <= y_max):
                    continue
                if (x1, y1) in searched:
                    continue
                    

                #print((x1, y1), (sgn_x * dx, sgn_y * dy), calc_dist(sensor, (x1, y1)))
                #assert calc_dist(sensor, (x1, y1)) == radius

                for _sensor, _dist in measurements:
                    if _sensor == sensor:
                        continue
                    if calc_dist(_sensor, (x1, y1)) <= _dist:
                        detected = True
                        searched.add((x1, y1))
                        break
                if not detected:
                    sol2 = x1 * 4000000 + y1
                    print("found the spot at", (x1, y1), "Solution=", sol2)
                    return sol2
                        

sol2 = solution2(test_data, 20, 20)

assert sol2 == 56000011
print("test passed")

found the spot at (14, 11) Solution= 56000011
test passed


## New solution for part 2 (did not finish with 1st)

[Inspiration from William Y. Feng](https://www.youtube.com/watch?v=w7m48_uCvWI) [Github](https://github.com/womogenes/AoC-2022-Solutions/blob/main/day_15/day_15_p2.py)

But new solution fails for test data as there is more than one point..

In [6]:
def solution2(data, lim):
    measurements = []
    for line in data.split("\n"):
        try: 
            sensor, beacon = [parse_xy(s) for s in line.split(":")]
        except Exception as e:
            print(line)
            raise e        
        dist = calc_dist(sensor, beacon)
        measurements.append([sensor, dist])

    for i1, sensor1 in enumerate(measurements):
        (x1, y1), dist1 = sensor1      
        # offset n for diagonal line with slope m=1 through sensor y = x + pos1
        pos1 = y1 - x1
        # offset n diagonal line with slope m=-1 through sensor y = x + neg1
        neg1 = y1 + x1 
        for i2, sensor2 in enumerate(measurements[i1+1:]):
            (x2, y2), dist2 = sensor2
            pos2 = y2 - x2
            neg2 = y2 + x2

            # check if borders are exactly 2 apart
            if abs(pos2 - pos1) == dist1 + dist2 + 2:
                # offset of that line between the two borders
                if pos1 > pos2:
                    pos = pos1 - dist1 - 1
                else:
                    pos = pos1 + dist1 + 1

            # same for above
            if abs(neg2 - neg1) == dist1 + dist2 + 2:
                if neg1 > neg2:
                    neg = neg1 - dist1 - 1
                else:
                    neg = neg1 + dist1 + 1

    # get coordinates of interception of the 2 lines
    x = (neg - pos) // 2
    y = x + pos
    assert (x <= lim) and (y <= lim)
    sol2 = x * 4000000 + y
    print("found beacon at:", (x, y))
    print("Result:", sol2)
    return sol2

lim = 4000000
sol2 = solution2(inp_data, lim)
assert sol2 == 13081194638237
print("test passed")

found beacon at: (3270298, 2638237)
Result: 13081194638237
test passed
